# 1 - Interacting with the Fitbit API

In this section we will be using the python-fitbit and the requests modules to get data from the Fitbit API. This is not the only way to do it, for example, a simple alternative would be to use the Fitbit Web API Explorer (https://dev.fitbit.com/build/reference/web-api/explore/). The steps taken here are largely outlined in this (https://towardsdatascience.com/using-the-fitbit-web-api-with-python-f29f119621ea) Towards Data Science article.

## 1.1 Setting up 

In this section we load all necessary modules, proceed with the authorization from the Fitbit API and also define the Fitbit object (from the python-fitbit module) which is used to make some GET requests to the Fitbit API.

In [1]:
# Import necessary modules
import gather_keys_oauth2 as Oauth2 # This is a python file you need to have in the same directory as your code so you can import it
import fitbit
import pandas as pd 
import datetime as dt
import requests as req
import warnings
import functions as fun

# Enter CLIENT_ID and CLIENT_SECRET
CLIENT_ID = '23QRRC'
CLIENT_SECRET = '51922a48a2df4434cc20afaac4ee97b8'

# Date after which we have data
START_DATE = "2023-05-02"
END_DATE = "2023-05-03"

Upon execution of the cell below, you will be redirected to another tab and will be asked to login into your Fitbit account. Upon doing that you will see a page that should say something like "Authentication Complete, you may close this tab".

In [2]:
# Authorize user
server = Oauth2.OAuth2Server(CLIENT_ID, CLIENT_SECRET)
server.browser_authorize()
# Save access and refresh tokens
ACCESS_TOKEN = str(server.fitbit.client.session.token['access_token'])
REFRESH_TOKEN = str(server.fitbit.client.session.token['refresh_token'])
EXPIRES_AT = str(server.fitbit.client.session.token['expires_at'])

[08/May/2023:20:40:06] ENGINE Listening for SIGTERM.
[08/May/2023:20:40:06] ENGINE Bus STARTING
CherryPy Checker:
The Application mounted at '' has an empty config.

[08/May/2023:20:40:06] ENGINE Set handler for console events.
[08/May/2023:20:40:06] ENGINE Started monitor thread 'Autoreloader'.
[08/May/2023:20:40:07] ENGINE Serving on http://127.0.0.1:8080
[08/May/2023:20:40:07] ENGINE Bus STARTED


127.0.0.1 - - [08/May/2023:20:40:09] "GET /?code=f613c0d1b9457745a1807ffa4a1ccbeb86a2b069&state=R8SualC8TTEWxI8VbzwTzYcNZWfuRC HTTP/1.1" 200 122 "" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36"


[08/May/2023:20:40:10] ENGINE Bus STOPPING
[08/May/2023:20:40:11] ENGINE HTTP Server cherrypy._cpwsgi_server.CPWSGIServer(('127.0.0.1', 8080)) shut down
[08/May/2023:20:40:11] ENGINE Stopped thread 'Autoreloader'.
[08/May/2023:20:40:11] ENGINE Removed handler for console events.
[08/May/2023:20:40:11] ENGINE Bus STOPPED
[08/May/2023:20:40:11] ENGINE Bus EXITING
[08/May/2023:20:40:11] ENGINE Waiting for child threads to terminate...
[08/May/2023:20:40:11] ENGINE Bus EXITED
[08/May/2023:20:40:11] ENGINE Waiting for thread Thread-16.


In the previous cells we completed the authorization process for the Id and Secret that is specified in the first code block. Next, we create an instance of the Fitbit object of the python-fitbit module, which will be the base object we will use to get the data we want.

In [3]:
# Create Fitbit object which will be used to get the data
auth2_client = fitbit.Fitbit(client_id = CLIENT_ID,
                             client_secret = CLIENT_SECRET,
                             expires_at = EXPIRES_AT,
                             oauth2 = True,
                             access_token = ACCESS_TOKEN,
                             refresh_token = REFRESH_TOKEN)

## 1.2 Get data

To get Sleep related data, we will be using the requests module. The reason for that is that for some reason in the python-fitbit module, there is an older API version variable that is hardcoded and we cannot change it (it's 1 while it should be 1.2 for Sleep data). As far as we know there is an issue about that in the module's Github page, but no implemented solution.

Therefore what we did is that we used the Fitbit Web API Explorer to get the CURL of the endpoint we want to draw data from, converted it to python using the requests module, and get the data we want.

For Activity data we used the python-fitbit module. We have used several different resources to quantify activity (steps, minutes active/sedentary).

Please keep in mind that there is a rate limit for each user who has consented to share their data, and this limit is **150 API requests per hour**. This resets at the top of each hour.


### Example code on how to get and transform sleep data: https://towardsdatascience.com/using-the-fitbit-web-api-with-python-f29f119621ea

# 2 - Import data into MongoDB

In [ ]:
#!pip install pymongo

In [4]:
import pymongo as mongo
client = mongo.MongoClient('localhost', 27017)

# Check if the connection to the db was successful
try:
    db = client.admin
    server_info = db.command('serverStatus')
    print('Connection to MongoDB server successful.')
    
except mongo.errors.ConnectionFailure as e:
    print('Connection to MongoDB server failed: %s' % e)


USER_UUID = "3cc4e2ee-8c2f-4c25-955b-fe7f6ffcbe44"
DB_NAME = "fitbit"
DATA_COLLECTION_NAME = "fitbitCollection"

Connection to MongoDB server successful.


In [6]:
client.drop_database("fitbit")

In [5]:
# Connect to the fitbitDb and the collection where the data are stored or create them if they don't exist
fitbitDb = client[DB_NAME]
fitbitCollection = fun.check_create_collection(fitbitDb, DATA_COLLECTION_NAME)
fitbitDb.list_collection_names()

# Define index
fitbitIndex = [('type', mongo.ASCENDING), ('data.dateTime', mongo.ASCENDING)]
fitbitIndexName = "type_1_data.dateTime_1"
fun.check_create_index(fitbitCollection, fitbitIndex, fitbitIndexName)


Collection fitbitCollection already exists, proceeding.
Index type_1_data.dateTime_1 already exists, proceeding.


## Import Sleep data to mongo

In this section we want to draw data from the Fitbit API and more specifically the Sleep endpoint, and then save that data in our local MongoDB instance.

Reference for what the keys mean:
https://dev.fitbit.com/build/reference/web-api/sleep/get-sleep-log-by-date/ 

In [7]:
global START_DATE
global END_DATE
global ACCESS_TOKEN

try:
    # Keys in the returned
    skipKeys = ["levels", "infoCode", "logId", "logType", "type", "dateOfSleep"]

    for single_date in fun.daterange(START_DATE, END_DATE):
        # Get data from the fitbit API
        oneDaySleepData = fun.get_sleep_data(single_date, ACCESS_TOKEN)

        # Check if sleep data exist for the date we are looking at
        if (len(oneDaySleepData["sleep"]) > 0):
            # Get data related to general sleep info as well as the sleep time series
            sleepData = oneDaySleepData["sleep"][0]
            # Define which keys we want to keep
            sleepDataKeys = [key for key in sleepData.keys() if key not in skipKeys or key == "levels"]
            # For each key containing general information on sleep
            for sleepDataKey in sleepDataKeys:  
                # Dictionary that will hold the 'data' entry of the document
                dataDict = {}    
                # 'levels' contains the time series data
                if sleepDataKey == "levels":
                    sleepLevelsData = sleepData[sleepDataKey]
                    for key in sleepLevelsData.keys():
                        if key == "data" or key == "shortData":
                            documentType = f"sleepLevelsData-{key}"
                            for dataPoint in sleepLevelsData[key]:
                                dataDict = {}
                                # Convert string date to datetime.datetime so that's saved correctly in mongo
                                dataDict["dateTime"] = fun.to_datetime(dataPoint["dateTime"])
                                dataDict["level"] = dataPoint["level"]
                                dataDict["value"] = dataPoint["seconds"]
                                fun.create_and_save_document(fitbitCollection, documentType, dataDict)
                else:
                    documentType = "sleep-{}".format(sleepDataKey)
                    # Convert datetime.date to datetime.datetime so that's saved correctly in mongo
                    dataDict["dateTime"] = fun.to_datetime(single_date)
                    if isinstance(sleepData[sleepDataKey], str):
                        dataDict["value"] = fun.to_datetime(sleepData[sleepDataKey])
                    else:
                        dataDict["value"] = sleepData[sleepDataKey]
                    fun.create_and_save_document(fitbitCollection, documentType, dataDict)
            # Get data related to summary sleep info
            sleepSummaryData = oneDaySleepData["summary"]
            # For each key containing summary information on sleep
            for sleepSummaryDataKey in sleepSummaryData.keys():
                documentType, dataDict = fun.get_summary_key_data(sleepSummaryData, sleepSummaryDataKey, single_date)
                fun.create_and_save_document(fitbitCollection, documentType, dataDict)
        else:
            warnings.warn(f"Could not find sleep data for {single_date}.")
            continue
        print(f'Loaded sleep data for {single_date}.')
except Exception as e:
    print('Error: %s' % e)



Loaded sleep data for 2023-05-02.


## Import Activity data to mongo


In [8]:
global START_DATE
global END_DATE

try:
    for single_date in fun.daterange(START_DATE, END_DATE):
        # Get activity data from the fitbit API
        oneDayActivityData = fun.get_activity_data(single_date, auth2_client)

        # For each kind of activity
        for activityTypeKey in oneDayActivityData.keys():
            for key in oneDayActivityData[activityTypeKey].keys():
                # Check if activity data exist for the date and type of activity we are looking at
                if len(oneDayActivityData[activityTypeKey][key]) > 0:
                    documentType = key.replace('activities-',"")
                    if "intraday" not in key:
                        dataDict = {}
                        dataDict["dateTime"] = fun.to_datetime(oneDayActivityData[activityTypeKey][key][0]["dateTime"])
                        dataDict["value"] = int(oneDayActivityData[activityTypeKey][key][0]["value"])
                        fun.create_and_save_document(fitbitCollection, documentType, dataDict)
                    else:
                        for dataPoint in oneDayActivityData[activityTypeKey][key]["dataset"]:
                            dataDict = {}
                            dataDict["dateTime"] = fun.to_datetime(single_date, time = dataPoint["time"])
                            dataDict["value"] = int(dataPoint["value"])
                            fun.create_and_save_document(fitbitCollection, documentType, dataDict)
                else:
                    warnings.warn(f"Could not find {activityTypeKey}-{key} data for {single_date}.")
                    continue 
        print(f'Loaded activity data for {single_date}.')                            
except Exception as e:
    print('Error: %s' % e)

Loaded activity data for 2023-05-02.
